# Libraries

In [ ]:
from PIL import Image
import os
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
tf.get_logger().setLevel(3)

# Building the Neural Network

In [ ]:
def create_cnn(width=154, height=154, depth=3, filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(32)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(16)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(11)(x)
    x = tf.keras.layers.Activation("softmax")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model  

# Preprocessing and converting the data to train and test

In [ ]:
base_path = 'dataset/'
data = {}
for classes in os.listdir(base_path):
    img_path = base_path+classes+'/'
    cls = int(classes)
    if  data.get(cls) is None:
        data[cls] = []

    for images in os.listdir(img_path):
        fmt = images.split('.')

        if fmt[-1] == "jpg":
            id = fmt[0]
            image_path = img_path + images
            img = Image.open(image_path)
            width, height = img.size
            top_crop = 66
            bottom_crop = 20
            left_crop = 43
            right_crop = 43 
            box = (left_crop, top_crop, width-right_crop, height - bottom_crop)
            cropped_img = img.crop(box)
            data[cls].append(np.array(cropped_img))
            img.close()

n_train = 4
Y_train = np.empty((0), int)
Y_test = np.empty((0), int)
X_train = np.empty((0,154,154,3), float)
X_test = np.empty((0,154,154,3), float)

for label in data:
    if label != 90:
        n = len(data[label])
        Y_train = np.append(Y_train, [label]*n_train, axis=0)
        X_train = np.append(X_train, data[label][0:n_train], axis=0)
        n -= n_train
        Y_test = np.append(Y_test, [label]*n, axis=0)
        X_test = np.append(X_test, data[label][n_train:], axis=0)
    else:
        dtr = np.array(data[label][0])
        dtr = dtr[np.newaxis,:]
        dte = np.array(data[label][1])
        dte = dte[np.newaxis,:]
        Y_train = np.append(Y_train, [label], axis=0)
        X_train = np.append(X_train, dtr, axis=0)
        Y_test = np.append(Y_test, [label], axis=0)
        X_test = np.append(X_test, dte, axis=0)

Y = np.zeros((11, len(Y_train)))
for i in range(0,len(Y_train)):
    Y[Y_train[i]//10,i] = 1

# Training the model

In [ ]:
model = create_cnn(154, 154, 3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=X_train, y=Y.T, epochs=200, batch_size=5, verbose=0)

# Testing the trained model

In [ ]:
preds = model.predict(X_test)
y_pred = np.zeros(len(preds))

for i in range(0,len(preds)):
    y_pred[i] = 10 * np.argmax(preds[i,:])

print("Confusion Matrix :")
print(confusion_matrix(Y_test, y_pred))
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(Y_test, y_pred)))